In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
path = './data/'
ll_data = 'leaf_level/group/'
csv = '.csv'

In [3]:
mkt_data = pd.read_csv(path+'power_market_MA'+csv, parse_dates = ["date"])
mkt_data.sort_values("date", inplace=True)
mkt_data.drop('Unnamed: 0', axis=1, inplace=True)
supp_data = pd.read_csv(path+ll_data+'Austru_85055_20000_Musharna'+csv, parse_dates = ["date"])
supp_data.sort_values("date", inplace=True)

In [4]:
supp_data = supp_data.groupby('date').agg({'price_year_eur':'mean'})

In [5]:
supp_data.head()

,price_year_eur
date,
2022-01-22,8440.0
2022-01-23,8440.0
2022-01-24,8440.0
2022-01-25,8170.0
2022-01-26,8612.0


In [6]:
supp_data.tail()

,price_year_eur
date,
2022-08-13,14758.000000
2022-08-14,14758.000000
2022-08-15,14758.000000
2022-08-16,14758.000000
2022-08-17,15412.666667


In [7]:
mkt_data.head()

,date,spot,base_d00,base_d01,base_d02,base_d03,base_d04,base_d05,base_d06,base_d07,...,peak_y06_SMA30,peak_y06_EWM30,peak_y06_SMA60,peak_y06_EWM60,peak_y06_SMA100,peak_y06_EWM100,peak_y06_SMA200,peak_y06_EWM200,peak_y06_SMA300,peak_y06_EWM300
0,2021-05-25,59.780417,59.78,65.61,68.10,73.73,57.80,51.66,69.22,70.92,...,67.212000,67.495835,65.541500,66.005817,64.0547,64.722294,62.12790,63.043412,61.572367,62.231843
1,2021-05-26,65.605833,65.61,71.77,78.24,64.67,49.43,75.34,75.46,70.92,...,67.372000,67.589652,65.645167,66.102347,64.1417,64.806011,62.17000,63.102184,61.601267,62.276485
2,2021-05-27,71.767500,71.77,73.60,62.36,46.94,73.16,75.76,74.76,70.92,...,67.541000,67.699997,65.754667,66.207188,64.2314,64.895001,62.21385,63.163854,61.629600,62.323157
3,2021-05-28,73.602083,73.60,56.49,43.15,67.88,72.33,71.49,60.90,70.92,...,67.699333,67.782578,65.858833,66.298100,64.3217,64.975892,62.25610,63.221726,61.657767,62.367391
4,2021-05-29,56.487917,56.49,37.85,67.46,68.25,64.08,59.83,69.75,57.18,...,67.843667,67.850154,65.953833,66.381113,64.4100,65.052211,62.29825,63.277529,61.685433,62.410335


In [8]:
mkt_data.tail()

,date,spot,base_d00,base_d01,base_d02,base_d03,base_d04,base_d05,base_d06,base_d07,...,peak_y06_SMA30,peak_y06_EWM30,peak_y06_SMA60,peak_y06_EWM60,peak_y06_SMA100,peak_y06_EWM100,peak_y06_SMA200,peak_y06_EWM200,peak_y06_SMA300,peak_y06_EWM300
502,2022-10-09,146.724583,146.72,162.09,259.87,301.67,250.00,198.72,134.28,96.72,...,145.944667,146.537960,150.134167,145.526188,144.5485,142.793117,134.93085,133.711244,122.402800,125.068876
503,2022-10-10,162.090000,162.09,259.87,301.67,250.00,198.72,134.28,96.72,96.72,...,146.551667,147.059382,150.179167,145.824346,144.6444,143.027313,135.20100,133.919291,122.601867,125.265230
504,2022-10-11,259.873750,259.87,282.54,255.62,225.96,141.96,74.96,185.42,96.72,...,147.106667,147.446519,150.216167,146.061581,144.7247,143.225980,135.46345,134.109746,122.802067,125.449913
505,2022-10-12,282.535833,282.54,257.46,236.92,176.00,89.56,221.58,224.39,96.72,...,147.656000,147.797711,150.301833,146.285463,144.8033,143.417347,135.72085,134.296614,123.002533,125.632240
506,2022-10-13,257.455833,257.46,260.65,173.98,98.02,214.71,246.97,227.71,96.72,...,147.767333,147.803020,150.304000,146.337743,144.8433,143.505716,135.95320,134.431772,123.185800,125.780066


In [9]:
supp_data = supp_data.merge(mkt_data, how='inner', on='date')

In [10]:
supp_data.corr()

C:\Users\Nutzer\AppData\Local\Temp\ipykernel_8240\437151006.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  supp_data.corr()


,price_year_eur,spot,base_d00,base_d01,base_d02,base_d03,base_d04,base_d05,base_d06,base_d07,...,peak_y06_SMA30,peak_y06_EWM30,peak_y06_SMA60,peak_y06_EWM60,peak_y06_SMA100,peak_y06_EWM100,peak_y06_SMA200,peak_y06_EWM200,peak_y06_SMA300,peak_y06_EWM300
price_year_eur,1.000000,0.601248,0.601250,0.603372,0.638909,0.669612,0.697823,0.682869,0.672073,0.683481,...,0.305202,0.331920,0.377527,0.391425,0.528737,0.446087,0.541969,0.499626,0.535629,0.518116
spot,0.601248,1.000000,1.000000,0.824265,0.682947,0.641341,0.598924,0.624707,0.746464,0.698315,...,0.405517,0.404316,0.465272,0.463419,0.567743,0.506657,0.571005,0.544431,0.568040,0.556536
base_d00,0.601250,1.000000,1.000000,0.824265,0.682949,0.641343,0.598925,0.624710,0.746465,0.698315,...,0.405518,0.404318,0.465273,0.463421,0.567745,0.506658,0.571006,0.544432,0.568041,0.556537
base_d01,0.603372,0.824265,0.824265,1.000000,0.843397,0.701593,0.638476,0.600092,0.647235,0.704093,...,0.413331,0.413986,0.473340,0.472549,0.575926,0.515712,0.580615,0.553646,0.577356,0.565855
base_d02,0.638909,0.682947,0.682949,0.843397,1.000000,0.845143,0.684171,0.634645,0.617456,0.698462,...,0.412906,0.412901,0.474117,0.473174,0.578556,0.517427,0.584395,0.556379,0.580682,0.568935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
peak_y06_EWM100,0.446087,0.506657,0.506658,0.515712,0.517427,0.512363,0.528827,0.552040,0.552023,0.626410,...,0.971799,0.974038,0.992970,0.995768,0.989691,1.000000,0.987680,0.996390,0.989000,0.993376
peak_y06_SMA200,0.541969,0.571005,0.571006,0.580615,0.584395,0.582299,0.599805,0.624105,0.627693,0.709776,...,0.930937,0.935231,0.969416,0.970795,0.995318,0.987680,1.000000,0.997143,0.999793,0.998893
peak_y06_EWM200,0.499626,0.544431,0.544432,0.553646,0.556379,0.552671,0.569794,0.593651,0.595614,0.674034,...,0.951183,0.954556,0.983567,0.984810,0.996867,0.996390,0.997143,1.000000,0.997849,0.999526
peak_y06_SMA300,0.535629,0.568040,0.568041,0.577356,0.580682,0.578172,0.595932,0.620517,0.624026,0.704532,...,0.932764,0.937588,0.971373,0.972604,0.996263,0.989000,0.999793,0.997849,1.000000,0.999348


In [11]:
supp_data.tail()

,date,price_year_eur,spot,base_d00,base_d01,base_d02,base_d03,base_d04,base_d05,base_d06,...,peak_y06_SMA30,peak_y06_EWM30,peak_y06_SMA60,peak_y06_EWM60,peak_y06_SMA100,peak_y06_EWM100,peak_y06_SMA200,peak_y06_EWM200,peak_y06_SMA300,peak_y06_EWM300
199,2022-08-13,14758.000000,377.745000,377.75,330.46,426.60,490.79,518.73,503.85,461.73,...,133.931000,135.995341,133.651667,134.149869,133.4578,131.041098,120.57500,121.861947,110.629733,113.798743
200,2022-08-14,14758.000000,330.455833,330.46,426.60,490.79,518.73,503.85,461.73,328.02,...,134.404000,136.753706,133.998167,134.595775,133.5890,131.371968,120.83750,122.119539,110.841067,114.024334
201,2022-08-15,14758.000000,426.604583,426.60,490.79,518.73,503.85,461.73,328.02,278.98,...,134.807000,137.463144,134.342833,135.027061,133.7522,131.696285,121.08700,122.374569,111.054067,114.248425
202,2022-08-16,14758.000000,490.792917,490.79,551.95,526.83,507.67,378.03,320.07,535.12,...,135.672667,139.022296,134.991333,135.899288,134.0542,132.289032,121.40215,122.765170,111.309500,114.563254
203,2022-08-17,15412.666667,551.952500,551.95,563.76,531.62,433.54,379.54,541.49,577.93,...,136.590333,140.581503,135.710333,136.794066,134.3718,132.900932,121.72185,123.167407,111.570133,114.886357


In [12]:
supp_data.iloc[:5, 300:330]

,base_wkd01_EWM200,base_wkd01_SMA300,base_wkd01_EWM300,base_wkd02_SMA7,base_wkd02_EWM7,base_wkd02_SMA15,base_wkd02_EWM15,base_wkd02_SMA30,base_wkd02_EWM30,base_wkd02_SMA60,...,base_q01_EWM15,base_q01_SMA30,base_q01_EWM30,base_q01_SMA60,base_q01_EWM60,base_q01_SMA100,base_q01_EWM100,base_q01_SMA200,base_q01_EWM200,base_q01_SMA300
0,126.494350,96.197900,109.130611,138.592857,127.828311,149.104667,138.684447,136.083000,143.341312,155.849000,...,168.731650,189.432000,189.881264,232.147333,205.506047,208.6820,201.079404,166.06710,173.134485,133.207333
1,127.073810,96.741867,109.632940,126.587143,119.163733,143.820000,132.995142,136.088667,140.104453,154.947667,...,170.425194,186.174667,189.390859,231.706000,204.744538,208.3913,200.707138,166.55675,173.225486,133.651367
2,127.647503,97.224500,110.131931,114.581429,112.665300,138.535333,128.016999,136.094333,137.076424,154.067167,...,171.907044,182.917333,188.932094,231.274500,204.007995,208.2316,200.342245,167.05665,173.315580,134.077267
3,127.229817,97.384233,109.969391,117.072857,124.358975,137.668667,131.944874,138.309000,138.519235,154.363000,...,174.791164,180.083333,189.322282,231.170833,203.711996,208.1989,200.236062,167.59450,173.531147,134.544033
4,126.776883,97.518767,109.781618,115.644286,125.519231,136.950000,131.576765,139.509000,137.905091,153.656000,...,177.319768,177.250667,189.689877,230.939833,203.427012,208.1666,200.132773,168.14070,173.744966,135.009367


In [13]:
new_mkt = supp_data.copy()

In [14]:
new_mkt.drop(columns = ['price_year_eur'], inplace = True)

In [15]:
new_mkt.set_index('date', inplace=True)

In [16]:
X = new_mkt

In [17]:
X

,spot,base_d00,base_d01,base_d02,base_d03,base_d04,base_d05,base_d06,base_d07,base_w00,...,peak_y06_SMA30,peak_y06_EWM30,peak_y06_SMA60,peak_y06_EWM60,peak_y06_SMA100,peak_y06_EWM100,peak_y06_SMA200,peak_y06_EWM200,peak_y06_SMA300,peak_y06_EWM300
date,,,,,,,,,,,,,,,,,,,,,
2022-01-22,175.757500,175.76,193.70,245.94,297.29,230.67,118.77,171.00,95.38,169.11,...,93.636667,93.490204,93.783333,92.611766,90.4767,90.456966,83.71875,84.973440,78.209667,80.752380
2022-01-23,193.695000,193.70,245.94,297.29,230.67,118.77,171.00,95.38,90.96,169.11,...,93.565000,93.465030,93.845833,92.627773,90.5542,90.509304,83.83560,85.054301,78.310900,80.834425
2022-01-24,245.937500,245.94,297.29,230.67,118.77,171.00,95.38,90.96,93.48,169.11,...,93.493333,93.441480,93.885000,92.643256,90.6417,90.560605,83.95260,85.134358,78.410800,80.915925
2022-01-25,297.288333,297.29,234.98,118.88,163.97,89.12,82.22,148.42,93.48,176.06,...,93.455000,93.483965,93.936667,92.691018,90.7392,90.630692,84.07085,85.223568,78.513033,81.003528
2022-01-26,234.981250,234.98,120.71,165.88,87.91,75.51,152.17,191.24,93.48,175.46,...,93.455000,93.597903,94.035000,92.774919,90.8482,90.722163,84.19385,85.323334,78.618033,81.098191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-13,377.745000,377.75,330.46,426.60,490.79,518.73,503.85,461.73,328.02,364.66,...,133.931000,135.995341,133.651667,134.149869,133.4578,131.041098,120.57500,121.861947,110.629733,113.798743
2022-08-14,330.455833,330.46,426.60,490.79,518.73,503.85,461.73,328.02,278.98,364.66,...,134.404000,136.753706,133.998167,134.595775,133.5890,131.371968,120.83750,122.119539,110.841067,114.024334
2022-08-15,426.604583,426.60,490.79,518.73,503.85,461.73,328.02,278.98,520.23,364.66,...,134.807000,137.463144,134.342833,135.027061,133.7522,131.696285,121.08700,122.374569,111.054067,114.248425


In [18]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [19]:
y = supp_data['price_year_eur']

In [20]:

pca = PCA(0.99)
X_pca = pca.fit_transform(X_scaled)
pca.n_components_



12

In [21]:
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [22]:
X_pca.shape

(204, 12)

In [23]:
y.shape

(204,)

In [24]:
# Split the data into training and test sets
#X = supp_data[['base_q04_SMA30']]
#y = supp_data['price_year_eur']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [25]:
# Train a random forest model
model = RandomForestRegressor(n_estimators=150, max_depth = 20, max_features='auto')
model.fit(X_train_pca, y_train)

C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


RandomForestRegressor(max_depth=20, max_features='auto', n_estimators=150)

In [26]:
# Test the model on the test data
predictions = model.predict(X_test_pca)

In [27]:
# Evaluate the model's performance
mae = np.mean(abs(predictions - y_test))
print(f'Mean absolute error: {mae:.2f}')

Mean absolute error: 153.18


In [28]:
relevant_instruments = [new_mkt.columns[i] for i in range(len(pca.components_))]

In [29]:
relevant_instruments

['spot',
 'base_d00',
 'base_d01',
 'base_d02',
 'base_d03',
 'base_d04',
 'base_d05',
 'base_d06',
 'base_d07',
 'base_w00',
 'base_w01',
 'base_w02']

In [30]:
# Use the model to make predictions on new data
new_energy_market_prices = [[210, 150, 165, 200, 185, 175, 180, 152], [240, 180, 195, 230, 220, 210 , 200, 190], [230, 170, 185, 220, 210, 230, 220, 215]]
predicted_supplier_prices = model.predict(new_energy_market_prices)
print(f'Predicted supplier prices: {predicted_supplier_prices}')

ValueError: X has 8 features, but RandomForestRegressor is expecting 12 features as input.

In [ ]:
model.score(X_test_pca, y_test)

In [ ]:
plt.plot(supp_data['date'], supp_data['price_year_eur'])
plt.show()

In [ ]:
plt.plot(supp_data['date'], supp_data['peak_y06_SMA100'])
plt.show()

In [ ]:
#df1 = supp_data.groupby('date').agg({'price_year_eur':'mean'})
#df1.join(X_test_pca).plot() 

In [ ]:
model = LogisticRegression()
model.fit(X_train_pca, y_train)

In [ ]:
predictions = model.predict(X_test_pca)

In [ ]:
mae = np.mean(abs(predictions - y_test))
print(f'Mean absolute error: {mae:.2f}')

In [ ]:
new_energy_market_prices = [[210, 150, 165, 200, 185], [240, 180, 195, 230, 220], [230, 170, 185, 220, 210]]
predicted_supplier_prices = model.predict(new_energy_market_prices)
print(f'Predicted supplier prices: {predicted_supplier_prices}')

In [ ]:
model.score(X_test_pca, y_test)